AI流程 数据处理 数据导入 模型构建 模型测试


In [ ]:
#/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Feature Extraction (FE)
重新生成 嵌入向量，也就是预处理咯
"""
import time
import h5py
import argparse

import torch
from transformers import T5EncoderModel,T5Tokenizer

#GPU 使用第一个gpu或cpu判定
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')
print("using divice {}".format(device))

using divice cuda:0


In [11]:
# 数据处理
def read_fasta(fasta_path):
    """
    读取 FASTA 文件，返回一个 dict：{uniprot_id: sequence}。
    header 行以 '>' 开始，取整行作为 ID，但将 '/' 和 '.' 替换为 '_'（因为 HDF5 dataset 名中这些字符可能有问题）。
    读取序列行时去掉所有空白并转为大写，移除 '-'（gap）。
    注意：如果同一个 header 出现多行序列，会拼接成一条
    """
    sequences = dict()
    with open(fasta_path,'r') as fasta:
        for line in fasta:
            if line.startswith('>'):#检查字符串是否以指定子字符串开头
                uniprot_id = line.replace('>','').strip() # 去除字符串首尾的空白字符
                uniprot_id = uniprot_id.replace('/','_').replace('.',"_")# 两种字符用_替代
                sequences[uniprot_id]=''#赋值给字典
            else: # 字典 赋值
                sequences[uniprot_id]+=''.join(line.split()).upper().replace("-","") # 字典，每行 去空 大写 替代
    return sequences
read_fasta("/home/ec2-user/project/test.fa")

{'cluster4_BIO|CRR141969_orf1998': 'MDKITTVNVGTEKDPQLVQIGSTLSLEERERLVALLKDFKDVFAWSYEDMPGINPEIVQHQIPLDPKARPIKQKLRRIRPDWAIKIKEEVTKQIDAGFLLVSEYLTWLANIVPVPKKDGKI',
 'cluster4_BIO|CRR141969_orf2000': 'VGTETAASTEMENATSIGIEGTASIGTFMPKPDRCLSAADGIWWEDDDLCLAHTDEDWKGNQPDDTWYIGEVDHMTRSGRYFKPPHLDQPEASGKDREAEKQKEKQLEDEAVLRQLKKIQADISIWGLLMASRVHRQAVLSAMDKAKLSIDTTPEQLVGLVFSRGATPMLTFSDKELPPEGANHNKPLYISVECRDKWIPVVLVDTGSAINICPARTAYAIGLKIADFVPTAQVIRAYDNTSREVMGTVKIQTKVGPGQHEIDFHVLEVPATFNLFLGRPWLHQVKAVSSTLHQVVKYPYGKGIAIVFRNLSIHPPPEVSTPVLEIEHGMEDVFISGFTLAEAWVVQDIMAANEGVYVSAQAVYLMNKLEHIAGMGLEKSGRKGVAALAEVPHNPHTFGLGYLPTKEDWIRKGK',
 'cluster4_BIO|CRR141873_orf239681': 'VRQKKAKNSVLISMHDVQSIESAERAYIGSLTVQPTLLGKVSQMYNLYEFLVSKLDDLVVDLIDDCPTDWSVDSDGGLWFKGRLGVPNIVELRKKIMDDTHRSRYTIHLGGTKIYHDLKRTFWWEGMKKDVGEYVSRCYVCQ'}

In [ ]:
# 模型构建
"""
加载模型，并缓存到指定目录，如果 device 是 CPU 则把模型 cast 到 float32，返回 model（eval 模式）和 vocab（tokenizer）
"""
def get_T5_model(model_dir, transformer_link = "Rostlab/prot_t5_xl_half_uniref50-enc"):
    print("Loading {}".format(transformer_link))
    if model_dir is not None:
        print("########################## model location ")
        print("Loading cached model from: {}".format(model_dir))
        print("##########################")
    
    model = T5EncoderModel.from_pretrained(transformer_link,cache_dir=model_dir)

    # only cast to full-precision if no GPU is available
    if device==torch.device("cpu"):
        model.to(torch.float32)
        #问题：为什么这样做？（GPU 靠低精度提性能、省资源；CPU 靠高精度保正确、保速度；同时用一套代码适配所有环境，降低维护成本。）现代 GPU（如 NVIDIA Ampere/Turing 架构）原生支持 FP16（半精度）、BF16 等低精度计算，低精度能大幅提升计算速度（吞吐量）、降低显存占用，且 GPU 的硬件浮点单元对低精度的数值稳定性有优化，因此无需强制转 FP32。也就是 从应用环境入手，低精度可以对大规模数据进行处理，但消耗资源多，因此发明 低精度；低精度是全精度的一半，无论是显存还是速度，但因cpu默认是32位，因此要把它转为FP32。（如果是x64架构呢？架构和精度不同，不能混淆）（这些名词是啥？https://zhuanlan.zhihu.com/p/673708074 ）
    
    # load module and token
    model = model.to(device).half()
    model = model.eval() #评估模式，no_grad()无梯度模式
    vocab = T5Tokenizer.from_pretrained(transformer_link,do_lower_case=False) #不强制转换大小
    return model,vocab

def get_embeddings(
    seq_path,
    model_dir,
    emb_path, #output H5 of embedding vector
    per_protein,#布尔值：True = 生成「整个蛋白的嵌入」（均值池化）；False = 保留「每个残基的嵌入」 (问题：True → 做均值池化（mean-pooling）：把所有残基的向量求平均，得到整个蛋白的 1 个向量 ; 不做池化，原样返回，形状是「序列长度 × 向量维度」，即每个残基 1 个向量)
    max_residues=4000,#单批次累计残基数上限（避免显存不够
    max_seq_len=4000,#序列长度阈值（超过则单独处理
    max_batch=1, #单批次最多序列
):
#报错：SyntaxError: non-default argument follows default argument 有默认参数的顺序
    """
    输入数据，进行运算。
    """
    seq_dict=dict()# 读取的数据保存
    emb_dict=dict()#最终的嵌入，用作调试
    # 1 加载序列 和 模型处理
    seq_dict=read_fasta(seq_path) #return sequences dict
    model,vocab=get_T5_model(model_dir) # load model and return model and vocab
    # test of output
    print('########################################')
    print('example sequence {}\n{}'.format(
    next(iter(seq_dict.keys())), 
    next(iter(seq_dict.values()))
    ))

    # 统计字典序列基础信息：平均长度、超长序列数量 []列表推导式
    avg_length = sum([len(seq) for _,seq in seq_dict.items()]) / len(seq_dict)
    n_long = sum([1 for _,seq in seq_dict.items() if len(seq)>max_seq_len])
    #按序列长度降序排序（先处理长序列，避免长序列扎堆导致OOM） 定位value，统计长度，排序
    sorted(seq_dict,key=lambda kv:len(kv[1]),reverse=True) #lambda 变量，计算

    print("Average sequence length: {}".format(avg_length))
    print("Number of sequences {}: {}".format(max_seq_len, n_long))

    ## 批量构建运行，避免OOM
    start = time.time()
    batch=list()# 临时存当前批次的序列（格式：[(ID, 空格分隔的序列, 长度), ...]）
    #遍历seq_dict，替换稀有字符，统计长度，加上空格，加入到batch中，计算当前批次累积残基数，处理批次
    for seq_idx,(pdb_id,seq) in enumerate(seq_dict.items(),1): #三个变量
        seq=seq.replace('U','X').replace('Z','X').replace('O','X')
        seq_len=len(seq)
        seq=' '.join(list(seq))
        batch.append((pdb_id,seq,seq_len))
        #print(batch)
        n_res_batch = sum([s_len for _,_,s_len in batch]) + seq_len #for循环导致最后一个seq_len没加 , n_res_batch作用：一共累加多少个残基

        # batch 处理
        # 1. 批次序列数≥max_batch；2. 累计残基数≥max_residues；3. 遍历到最后一个序列；4. 当前序列是超长序列（单独处理）
        if len(batch)>=max_batch or n_res_batch>=max_residues or seq_idx==len(seq_dict) or seq_len>max_seq_len:
            # 拆分批次：把batch里的ID、序列、长度分别拆成三个元组
            pdb_ids,seqs,seq_lens = zip(*batch) # 转制变量
            batch = list()
            #以上，序列处理完成，序列进行embedding和 training model

            ########## 分词器编码：把序列转成模型能识别的数字ID ##########
            # batch_encode_plus：批量编码；add_special_tokens=True：加T5的特殊token（如<s>/</s>）；padding="longest"：按批次最长序列补零
            token_encoding = vocab.batch_encode_plus(seqs,add_apecial_tokens=True,padding="longest")
            input_ids = torch.tensor(token_encoding['input_ids']).to(device)
            attention_mask = torch.tensor(token_encoding['attention_mask']).to(device)

            ########### 模型推理：生成嵌入 ##########
            #模型推理，失败跳过
            try:
                with torch.no_grad():
                    embedding_repr = model(input_ids,attention_mask=attention_mask)
            except:
                print("RuntimeError during embedding for {} (L={}). Try lowering batch size. ".format(pdb_id, seq_len) + "If single sequence processing does not work, you need more vRAM to process your protein.")
                continue

            ## 4 嵌入后处理：切分 padding + 池化，存到字典
            # ########## 嵌入处理：去掉padding，可选均值池化 ##########
            # embedding_repr.last_hidden_state.shape = [批次大小, 序列长度（含padding）, 嵌入维度]
            # 遍历批次内的每个序列
            for batch_idx, identifier in enumerate(pdb_ids):
                s_len = seq_lens[batch_idx]#真实长度
                emb=embedding_repr.last_hidden_state[batch_idx,:s_len]#保留这个长度

                # 如果需要蛋白级嵌入（per_protein=True），对残基嵌入做均值池化（dim=0：按序列长度维度求平均）
                if per_protein:
                    emb = emb.mean(dim=0)
                # 打印第一个嵌入的形状（方便调试）
                if len(emb_dict) == 0:
                    print("Embedded protein {} with length {} to emb. of shape: {}".format(identifier, s_len, emb.shape))
                #把张量从 GPU 挪到 CPU，转成 numpy 数组（脱离 PyTorch 计算图），避免显存泄漏。
                emb_dict[identifier]=emb.detach().cpu().numpy().squeeze()
    end = time.time()
    with h5py.File(str(emb_path),'w') as hf:
        for sequence_id,embedding in emb_dict.items():
            hf.create_dataset(sequence_id, data=embedding)
    # 打印统计信息（方便评估效率）
    print('\n############# STATS #############')
    print('Total number of embeddings: {}'.format(len(emb_dict)))
    print('Total time: {:.2f}[s]; time/prot: {:.4f}[s]; avg. len= {:.2f}'.format( 
            end-start, (end-start)/len(emb_dict), avg_length))
    return True


#get_embeddings(seq_path="/home/ec2-user/project/test.fa",emb_path="/home/ec2-user/project/emb",model_dir="/home/ec2-user/project/model",per_protein=False)

序列embedding需要考虑的几个方面：  
1 序列过长：更细致的 fallback（比如分段或 sliding window）  
2 保存模型：占用大量显存。可以考虑更细粒度控制（比如按长度分桶）。

In [ ]:
# 模型测试
def create_arg_parser():
    """
    Using argparse to running for this py coding
    """
    parser = argparse.ArgumentParser(description=('t5_embedder.py creates T5 embeddings for a given text '+' file containing sequence(s) in FASTA-format.'))
    parser.add_argment("-i","--input",required=True,type=str,help='A path to a fasta-formatted text file containing protein sequence(s).')
    parser.add_argment('-o','--output',required=True,type=str,help='A path for saving the created embeddings as NumPy npz file.')
    parser.add_argment('--model',required=False,type=str,help='A path to a directory holding the checkpoint for a pre-trained model')
    parser.add_argment('--per_protein',type=int,default=0,help="Whether to return per-residue embeddings (0: default) or the mean-pooled per-protein representation (1).")
    return parser

def main():
    parser = create_arg_parser()
    args = parser.parse_args()

    seq_path = Path(args.input)
    emb_path = Path(args.output)
    model_dir = Path(args.model) if args.model is not None else None
    per_protein = False if int(args.per_protein) == 0 else True

if __name__ == "__main__":
    main()

AttributeError: 'ArgumentParser' object has no attribute 'add_argment'

# 补充
数据是怎么流动？